In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt

# Check for TensorFlow GPU access
print(tf.config.list_physical_devices())

# See TensorFlow version
print(tf.__version__)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.16.1


In [3]:


# Load data
data = pd.read_csv('eth_usd_candles.csv', parse_dates=['time'], index_col='time')
data = data[['close']]  # We will use the 'close' price for prediction

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences
def create_sequences(data, sequence_length):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:i + sequence_length]
        y = data[i + sequence_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 60  # Use the past 60 data points to predict the next one
X, y = create_sequences(scaled_data, sequence_length)

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [4]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=1)

# Save the model
model.save('eth_lstm_model.h5')


2024-06-01 19:07:50.624671: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-06-01 19:07:50.624695: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2024-06-01 19:07:50.624700: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2024-06-01 19:07:50.624719: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-01 19:07:50.624733: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models,

  5127/119619 ━━━━━━━━━━━━━━━━━━━━ 34:21 18ms/step - loss: 4.6428e-04

KeyboardInterrupt: 

In [ ]:
# Load the model (if needed)
# from tensorflow.keras.models import load_model
# model = load_model('eth_lstm_model.h5')

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Inverse transform to get actual values

# Plot the results
plt.figure(figsize=(16, 8))
plt.plot(data.index, data['close'], label='Actual')
plt.plot(data.index[split + sequence_length:], predictions, label='Predicted')
plt.legend()
plt.show()
